# Obligatorio 2 - Big Data Science

Integrantes del grupo: [completar aquí los nombres de los integrantes]

Debajo de cada pregunta o tarea incluya las celdas necesarias para desarrolar la respuesta. Puede usar una o varias celdas de código o mark down (https://www.datacamp.com/community/tutorials/markdown-in-jupyter-notebook)

Para entregar, renombrar este notebook como "Obligatorio 2 - Apellido1 - Apellido 2 - Apellido 3" con los apellidos de los miembros del grupo. Un solo integrante del grupo debe realizar la entrega. 


1. Cargar los datos del Obligatorio 1, de entrenamiento (.data) y validación (.test) en spark dataframes (distintos). Los nombres de las columnas deben corresponder a los especificados en "Attribute Information".

In [1]:
from pyspark.context import SparkContext
sc = SparkContext('local', 'test')

In [2]:
sc

<SparkContext master=local appName=test>

In [3]:
from pyspark.sql import SQLContext

spark = SQLContext(sc)

In [4]:
spark

In [5]:
from pyspark.sql.types import *

# Schema de los dataset
schema = StructType([
    StructField(name='age', dataType=IntegerType(), nullable=False),
    StructField(name='workclass', dataType=StringType(), nullable=False),
    StructField(name='fnlwgt', dataType=StringType(), nullable=False), # Se pone en tipo string porque al ponerlo en tipo Integer o Long carga la columna con valores null
    StructField(name='education', dataType=StringType(), nullable=False),
    StructField(name='education_num', dataType=StringType(), nullable=False), # Mismo problema que en la columna 'fnlwgt'
    StructField(name='marital-status', dataType=StringType(), nullable=False),
    StructField(name='occupation', dataType=StringType(), nullable=False),
    StructField(name='relationship', dataType=StringType(), nullable=False),
    StructField(name='race', dataType=StringType(), nullable=False),
    StructField(name='sex', dataType=StringType(), nullable=False),
    StructField(name='capital_gain', dataType=StringType(), nullable=False), # Mismo problema que en la columna 'fnlwgt'
    StructField(name='capital_loss', dataType=StringType(), nullable=False), # Mismo problema que en la columna 'fnlwgt'
    StructField(name='hours_per_week', dataType=StringType(), nullable=False), # Mismo problema que en la columna 'fnlwgt'
    StructField(name='native_country', dataType=StringType(), nullable=False),
    StructField(name='income', dataType=StringType(), nullable=False)
])

In [6]:
df_adult = spark.read.csv("adult.data", header=True, schema=schema)
df_test = spark.read.csv("adult.test", header=True, schema=schema)

In [7]:
df_adult.show(10)

+---+-----------------+-------+-------------+-------------+--------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
|age|        workclass| fnlwgt|    education|education_num|      marital-status|        occupation|  relationship|  race|    sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+---+-----------------+-------+-------------+-------------+--------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
| 50| Self-emp-not-inc|  83311|    Bachelors|           13|  Married-civ-spouse|   Exec-managerial|       Husband| White|   Male|           0|           0|            13| United-States| <=50K|
| 38|          Private| 215646|      HS-grad|            9|            Divorced| Handlers-cleaners| Not-in-family| White|   Male|           0|           0|            40| United-States| <=50K|
| 53|          Private| 234721|    

In [8]:
df_test.show(10)

+---+-----------------+-------+-------------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+-------+
|age|        workclass| fnlwgt|    education|education_num|     marital-status|        occupation|  relationship|  race|    sex|capital_gain|capital_loss|hours_per_week|native_country| income|
+---+-----------------+-------+-------------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+-------+
| 25|          Private| 226802|         11th|            7|      Never-married| Machine-op-inspct|     Own-child| Black|   Male|           0|           0|            40| United-States| <=50K.|
| 38|          Private|  89814|      HS-grad|            9| Married-civ-spouse|   Farming-fishing|       Husband| White|   Male|           0|           0|            50| United-States| <=50K.|
| 28|        Local-gov| 336951|   A

2. Seleccionar un conjunto relevante de 5 atributos y crear un Spark Pipeline en el que el estimator sea un DecisionTreeClassifier (https://spark.apache.org/docs/latest/ml-classification-regression.html#decision-tree-classifier). Puede utilizar libremente los transformers/estimators de Spark para realizar ingeniería de atributos (StringIndexer, OneHotEncoding, etc).

In [17]:
atributos = ["age", "sex", "occupation", "education", "workclass"]

#Primero hay que bucketizar la edad. La separamos en 3 buckets. 

def calculo_de_buckets(df, cantidad_buckets, columna):
    df.createOrReplaceTempView("df_view")
    instancias_totales = spark.sql('''SELECT count(*) as count FROM df_view''').first()['count']
    instancias_en_bucket = instancias_totales / cantidad_buckets
    
    limites = []
    ultimo_limite_superior = None
    for limit in range(0, instancias_totales - 1, round(instancias_en_bucket)): # se pone un -1 porque instancias totales es el count pero queremos llegar hasta el indice        
        query = spark.sql(f'''SELECT {columna} FROM df_view ORDER BY 1 asc''').collect()
        limite_inferior = query[limit][columna]
        
        if(ultimo_limite_superior is not None):
            if ultimo_limite_superior == limite_inferior:
                limite_inferior += 1;
        
        limite = [limite_inferior, query[limit + round(instancias_en_bucket)][columna]]
        limites.append(limite)
        
        ultimo_limite_superior = query[limit + round(instancias_en_bucket)][columna]

    return limites

limites = calculo_de_buckets(df_adult, 3, "age")

In [15]:
from pyspark.sql import functions as F

def age_bucket_udf(age):
    indexes = range(len(limites))
    for index, limite in zip(indexes, limites):
        if age in range(limite[0], limite[1] + 1):    #Se agrega el +1 para hacer el range inclusivo superiormente
            return index

age_udf = F.udf(age_bucket_udf, IntegerType())

df_adult_bucketized = df_adult.withColumn("ageLabel", age_udf(F.col("age")))
df_adult_bucketized.show()

+---+-----------------+-------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+----------+
|age|        workclass| fnlwgt|    education|education_num|      marital-status|        occupation|  relationship|               race|    sex|capital_gain|capital_loss|hours_per_week|native_country|income|age_bucket|
+---+-----------------+-------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+----------+
| 50| Self-emp-not-inc|  83311|    Bachelors|           13|  Married-civ-spouse|   Exec-managerial|       Husband|              White|   Male|           0|           0|            13| United-States| <=50K|         2|
| 38|          Private| 215646|      HS-grad|            9|            Divorced| Handlers-cleaners| Not-in-family|              Whit

In [118]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler

In [165]:
def get_indexers(columnas):
    
    list_indexer = []
    
    for columna in columnas:
        list_indexer.append(StringIndexer(inputCol=columna, outputCol=columna+"Label"))
    
    return list_indexer

In [186]:
cols = ["sex", "occupation", "education", "workclass"]
features_col = 'income'

labels_cols = []
for col in cols:
    labels_cols.append(col+"Label")

indexersLabels = get_indexers(cols)

featureIndexer = VectorIndexer().setInputCol(features_col).setOutputCol(features_col+"Feature")

assembler = VectorAssembler(
    inputCols=labels_cols,
    outputCol="features")

In [187]:
data = indexerFeatures.fit(df_adult).transform(df_adult)
data.show(10)

+---+-----------------+-------+-------------+-------------+--------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+--------------+
|age|        workclass| fnlwgt|    education|education_num|      marital-status|        occupation|  relationship|  race|    sex|capital_gain|capital_loss|hours_per_week|native_country|income|incomeFeatures|
+---+-----------------+-------+-------------+-------------+--------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+--------------+
| 50| Self-emp-not-inc|  83311|    Bachelors|           13|  Married-civ-spouse|   Exec-managerial|       Husband| White|   Male|           0|           0|            13| United-States| <=50K|           0.0|
| 38|          Private| 215646|      HS-grad|            9|            Divorced| Handlers-cleaners| Not-in-family| White|   Male|           0|           0|            4

In [188]:
# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="features", featuresCol=features_col+"Features")

In [192]:
# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=indexersLabels + [featureIndexer] + [dt])

3. Hacer el fit del pipeline con los datos de entrenamiento. 


In [193]:
trainingData = df_adult_bucketized
testData = df_test

In [194]:
# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

IllegalArgumentException: requirement failed: Column income must be of type class org.apache.spark.ml.linalg.VectorUDT:struct<type:tinyint,size:int,indices:array<int>,values:array<double>> but was actually class org.apache.spark.sql.types.StringType$:string.

In [172]:
# Make predictions.
predictions = model.transform(testData)

In [173]:
predictions.show(100)

+---+-----------------+-------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+-------------------+-------+--------+---------------+--------------+--------------+-------------------+-------------+-----------+----------+
|age|        workclass| fnlwgt|    education|education_num|      marital-status|        occupation|  relationship|               race|    sex|capital_gain|capital_loss|hours_per_week|     native_country| income|sexLabel|occupationLabel|educationLabel|workclassLabel|     incomeFeatures|rawPrediction|probability|prediction|
+---+-----------------+-------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+-------------------+-------+--------+---------------+--------------+--------------+-------------------+-------------+-----------+----------+
| 25|          Private| 2268

In [ ]:
#

4. Graficar la curva ROC utilizando los datos de validación (sin usar el paquete de evluación de Spark pyspark.ml.evaluation)

In [6]:
#

5. Escribir dos funciones train() y predict() que creen el arbol de acuerdo a la metodología vista en clase (utilizando entropía como métrica de homogeneidad de clases).
```
def train(train_dataframe):
    '''
    @return devuelve una estructura de datos que representa el árbol de decision
    '''
     pass        
```

```
def predict(tree, train_dataframe)
    '''
    @param tree la estructura de datos que representa el árbol de decisión.
    @ return un dataframe con todos los datos de train_dataframe con una columna adicional que representa la probabilidad de que el income sea >50K. 
    '''
    pass
```    
Ejemplo de uso:

```
tree = train(train_dataframe)
predictions_df = predict(tree, train_dataframe)
```

In [7]:
# 

<i>[Extra credit y alumnos masters]</i>
    
5.  Mejorar la implementación de su algoritmo evitando hacer el split cuando no se logra un mínimo de Information Gain ( https://en.wikipedia.org/wiki/Information_gain_in_decision_trees   )


In [8]:
#